In [1]:
#hide
#default_exp doclinks

# nbdev.doclinks
- Generating a documentation index from a module

In [2]:
#export
from nbdev.read import *
from nbdev.export import *
from nbdev.imports import *
from fastcore.script import *
from fastcore.imports import *
from fastcore.utils import *

import ast,contextlib
from pprint import pformat
from urllib.parse import urljoin

In [3]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload

In [4]:
#export
def _mod_fn2name(fn):
    "Convert filename `fn` to its module name"
    return '.'.join(str(Path(fn).with_suffix('')).split('/'))

class DocLinks:
    "Create a module symbol index from a python source file"
    def __init__(self, mod_fn, doc_func, dest_fn, mod_name=None):
        mod_fn,dest_fn = Path(mod_fn),Path(dest_fn)
        if mod_name is None: mod_name = _mod_fn2name(
            mod_fn.resolve().relative_to(dest_fn.parent.parent.resolve()))
        store_attr()
        if self.dest_fn.exists(): self.d = exec_local(self.dest_fn.read_text(), 'd')
        else: self.d = dict(syms={}, settings={}) 

The doc index has to be stored in a file. Usually we call it `_modidx.py`. For testing, we'll delete any existing file first.

In [5]:
dest_fn = Path('tmp/_modidx.py')
with contextlib.suppress(FileNotFoundError): dest_fn.unlink()

A link to docs is created by a `doc_func`. We'll use a dummy one for testing.

In [6]:
def _help(m, s=None): return f"help for {m}; {s}"

We're now ready to instantiate `DocLinks` for our test module.

In [7]:
mod_fn = Path('tmp/everything.py')
link = DocLinks(mod_fn, _help, dest_fn)
link.mod_name

'tmp.everything'

In [8]:
#export
@patch
def write_nbdev_idx(self:DocLinks):
    "Create nbdev documentation index file`"
    self.dest_fn.write_text("# Autogenerated by nbdev\n\nd = "
                            + pformat(self.d, width=160, indent=2, compact=True, sort_dicts=False))

Initially the index file will be empty, except for a comment noting that it's auto-generated.

In [9]:
link.write_nbdev_idx()
assert "Autogenerated" in dest_fn.read_text()

In [10]:
#export
def all_or_exports(code):
    parsed = ast.parse(code)
    res = read_var(code, '__all__')
    return L(retr_exports(L(parsed.body)) if res is None else res),parsed

In [11]:
#export
def _is_patch(o): return any(L(o.decorator_list).filter(Self.id.startswith('patch')))
def _pat_name(o):
    try: return f'{o.args.args[0].annotation.id}.{o.name}'
    except AttributeError: return None 

def _exp_meths(tree):
    return L(f"{tree.name}.{o.name}" for o in tree.body
             if isinstance(o,(ast.FunctionDef,ast.AsyncFunctionDef)) and o.name[0]!='_')

In [12]:
#export
@patch
def update_syms(self:DocLinks):
    code = Path(self.mod_fn).read_text()
    exp,parsed = all_or_exports(code)
    trees = L(parsed.body)
    exp_class = trees.filter(lambda o: isinstance(o, ast.ClassDef) and o.name in exp)
    exp += exp_class.map(_exp_meths).concat()
    pats = L(f'{o.args.args[0].annotation.id}.{o.name}' for o in trees
       if isinstance(o,(ast.FunctionDef,ast.AsyncFunctionDef)) and _is_patch(o))
    exp += pats.filter()
    exp = exp.map(f"{self.mod_name}.{{}}")
    self.d['syms'][self.mod_name] = exp.map_dict(partial(self.doc_func, self.mod_name))

In [13]:
everything_fn = '../tests/01_everything.ipynb'
ExportModuleProcessor('../tests/00_some.thing.ipynb', 'tmp').create_modules()
proc = ExportModuleProcessor(everything_fn, 'tmp')
proc.create_modules()

In [14]:
link.update_syms()
link.write_nbdev_idx()

import tmp._modidx
reload(tmp._modidx)
d = tmp._modidx.d
symn = 'tmp.everything.a_y'
mod_name = 'tmp.everything'
test_eq(d['syms'][mod_name][symn], _help(mod_name,symn))
test_eq(set(d['syms'][mod_name].keys()),
        set(L('m_y', 'n_y', 'q_y', 'a_y', 'b_y', 'd_y', 'e_y', 'o_y', 'p_y', 'd_y.di_n', 'd_y.d3i_n'
             ).map('tmp.everything.{}')))

In [15]:
#export
@patch
def build_index(self:DocLinks):
    self.update_syms()
    self.d['settings'] = dict(**Config().d)
    self.write_nbdev_idx()

In [16]:
link.build_index()
link.write_nbdev_idx()
reload(tmp._modidx)
test_eq(tmp._modidx.d['settings']['lib_name'], 'nbdev')

## CLI

- write tmpls

In [17]:
#export
def _doc_link(url, mod, sym=None):
    res = urljoin(url, mod)
    if sym: res += "#" + remove_prefix(sym, mod+".")
    return res

In [18]:
#export
def _update_baseurl(path=None):
    "Add or update `baseurl` in `_config.yml` for the docs"
    _re_baseurl = re.compile('^baseurl\s*:.*$', re.MULTILINE)
    path = Path(ifnone(path, Config().doc_path))
    fname = path/'_config.yml'
    if not fname.exists(): return
    code = fname.read_text()
    if _re_baseurl.search(code) is None: code = code + f"\nbaseurl: {Config().doc_baseurl}"
    else: code = _re_baseurl.sub(f"baseurl: {Config().doc_baseurl}", code)
    fname.write_text(code)

def _use_nb(p): return not p.name.startswith('_') and '.ipynb_checkpoints' not in p.parts

In [19]:
#export
@call_parse
def nbdev_build_lib(
    nbs:Param("Glob specifiying notebooks to export (defaults to all nbs in `nbs_path`)", str)=None,
    dest:Param("Destination for library (defaults to `lib_path`)", str)=None):
    "Convert notebooks matching `nbs` to modules"
    cfg = Config()
    dest = cfg.config_path/(ifnone(dest, cfg.lib_path))
    if os.environ.get('IN_TEST',0): return
    _fn = dest/'_modidx.py'
    if nbs is None:
        files = L(cfg.path('nbs_path').glob('*.ipynb')).filter(_use_nb)
        with contextlib.suppress(FileNotFoundError): _fn.unlink()
    else: files = glob.glob(nbs)
    for file in files: ExportModuleProcessor(file, dest).create_modules()
    doc_func = partial(_doc_link, urljoin(cfg.doc_host,cfg.doc_baseurl))
    for file in dest.glob("**/*.py"):
        if file.name[0]!='_': DocLinks(file, doc_func, _fn).build_index()
    if not cfg.get('extension',False):
        _update_baseurl()
        add_init(dest)

## Export -

In [20]:
nbdev_build_lib()

from nbdev import __version__
from nbdev.read import *
import nbdev.export
reload(nbdev.export)
assert __version__
assert hasattr(nbdev.export, 'read_nb')